# Jupyter Notebook to clean install all the contracts

# Set Up Environment

In [8]:
network = "yeouido"  # set this to one of mainnet, yeouido, euljiro, pagoda, or custom
contracts_file_custom = 'contracts_20201108141815.pkl'
#contracts_20201109120425

connections = {
"mainnet": {"iconservice": "https://ctz.solidwallet.io",       "nid": 1},
"yeouido": {"iconservice": "https://bicon.net.solidwallet.io", "nid": 3},
"euljiro": {"iconservice": "https://test-ctz.solidwallet.io",  "nid": 2},
"pagoda":  {"iconservice": "https://zicon.net.solidwallet.io", "nid":80},
"custom":  {"iconservice": "http://18.237.205.52:9000/",       "nid": 3},
"local":   {"iconservice": "http://localhost:9000/",           "nid": 3}
}

env = connections[network]

In [2]:
class AutoVivification(dict):
    """Implementation of perl's autovivification feature."""
    def __getitem__(self, item):
        try:
            return dict.__getitem__(self, item)
        except KeyError:
            value = self[item] = type(self)()
            return value
            

In [3]:
from iconsdk.exception import JSONRPCException
from iconsdk.libs.in_memory_zip import gen_deploy_data_content
from iconsdk.icon_service import IconService
from iconsdk.providers.http_provider import HTTPProvider
from iconsdk.builder.transaction_builder import CallTransactionBuilder, TransactionBuilder, DeployTransactionBuilder
from iconsdk.builder.call_builder import CallBuilder
from iconsdk.signed_transaction import SignedTransaction
from iconsdk.wallet.wallet import KeyWallet
from iconsdk.utils.convert_type import convert_hex_str_to_int
from datetime import datetime
from repeater import retry
from shutil import make_archive
import requests
import json
import pickle as pkl
import csv
import os
from pprint import pprint
from getpass import getpass


ICX = 1000000000000000000 # 10**18
GOVERNANCE_ADDRESS = "cx0000000000000000000000000000000000000000"

with open(contracts_file_custom, 'rb') as f:
    contracts = pkl.load(f)

@retry(JSONRPCException, tries=10, delay=1, back_off=2)
def get_tx_result(_tx_hash):
    tx_result = icon_service.get_transaction_result(_tx_hash)
    return tx_result




### Required service

In [9]:
icon_service = IconService(HTTPProvider(env["iconservice"], 3))
NID = env["nid"]

# Importing wallet

### Importing from private key and printing ICX balance

In [99]:
deployer_wallet4 = KeyWallet.create()
deployer_wallet4.get_private_key()

'287397cae97a02931e9e6f961ac79706b7f605e850d9f34ab10f99d78091eda0'

In [100]:
private1="a691ef7d5601f9b5be4f9b9d80215159ea6ff0b88003e3d34e078d40e778b39a"
deployer_wallet1 = KeyWallet.load(bytes.fromhex(private1))
print(deployer_wallet1.get_address())

private2="94a1471d2fa8114f585bb0c7baca343b2d57d2c65b2d9c0f461da828e3741f04"
deployer_wallet2 = KeyWallet.load(bytes.fromhex(private2))
print(deployer_wallet2.get_address())

private3="d7776d1ae9103bd49eeed8463e4ce50480b1f4bc760108325415998c239fcee5"
deployer_wallet3 = KeyWallet.load(bytes.fromhex(private3))
print(deployer_wallet3.get_address())


private4="f569eb1cbc8a34f2d36f7b6f569294fb1a0fada68c866250b38d1e6165889855"
deployer_wallet4 = KeyWallet.load(bytes.fromhex(private4))
print(deployer_wallet4.get_address())


private5="287397cae97a02931e9e6f961ac79706b7f605e850d9f34ab10f99d78091eda0"
deployer_wallet5 = KeyWallet.load(bytes.fromhex(private5))
print(deployer_wallet5.get_address())




hx91bf040426f226b3bfcd2f0b5967bbb0320525ce
hx8c8ccc79c99375de5995ca6057cda7c334d9dd35
hx49c8a4c31e84b9abe7f8c8395eb7133b2151aeae
hxd4ff18afb94cd6cb41d8ff468254112619a19051
hx5bb03cc3f3d26b2d27491c2f56d233ed6777a654


In [101]:
transaction = TransactionBuilder()\
    .from_(deployer_wallet1.get_address())\
    .to(deployer_wallet4.get_address())\
    .nid(3) \
    .nonce(100) \
    .value(500*10**18)\
    .build()

estimate_step = icon_service.estimate_step(transaction)
step_limit = estimate_step + 100000
# Returns the signed transaction object having a signature
signed_transaction = SignedTransaction(transaction, deployer_wallet1,step_limit)

# Reads params to transfer to nodes
print(signed_transaction.signed_transaction_dict)

# Sends the transaction
tx_hash = icon_service.send_transaction(signed_transaction)

get_tx_result(tx_hash)

{'version': '0x3', 'from': 'hx91bf040426f226b3bfcd2f0b5967bbb0320525ce', 'to': 'hxd4ff18afb94cd6cb41d8ff468254112619a19051', 'stepLimit': '0x30d40', 'timestamp': '0x5b61ef407ff4c', 'nid': '0x3', 'value': '0x1b1ae4d6e2ef500000', 'nonce': '0x64', 'signature': 'Akf4OAh35W9GKtb7Pcm+gRAndsHMXx8vHlpVdB/k6yInCyPrl7+yhZYrNr3iNt6pXcvKXGJOQ5Cxsi5r4HNAMAE='}


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...


{'txHash': '0x166e4204b9c182a3a915c9b2b045fa5242850eb058021dcdc464af630979b771',
 'blockHeight': 10289153,
 'blockHash': '0xac874e17ffd32930098052dc507c4564027a417a52d7d81fa2279d297d3f7707',
 'txIndex': 1,
 'to': 'hxd4ff18afb94cd6cb41d8ff468254112619a19051',
 'stepUsed': 100000,
 'stepPrice': 10000000000,
 'cumulativeStepUsed': 100000,
 'eventLogs': [],
 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00

In [102]:

params = {"_to": deployer_wallet5.get_address(), "_value": 500*10**18}
transaction = CallTransactionBuilder()\
    .from_(deployer_wallet1.get_address())\
    .to(contracts['sample_token']['SCORE'])\
    .value(0)\
    .step_limit(10000000)\
    .nid(3)\
    .nonce(100)\
    .method("transfer")\
    .params(params)\
    .build()
signed_transaction = SignedTransaction(transaction, deployer_wallet1)
tx_hash = icon_service.send_transaction(signed_transaction)
get_tx_result(tx_hash)

Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...


{'txHash': '0x4c99e77d6494d73566c0780a3bafb60d05c703b40f650f5e890bf6df909b0973',
 'blockHeight': 10289158,
 'blockHash': '0x72f0d5d52df971bd5869cf77cefa16cce34b0a13ddf02b0c353b43e5d38ff661',
 'txIndex': 1,
 'to': 'cx19be1be7b4e7750863edf966faac3f42dec21025',
 'stepUsed': 160840,
 'stepPrice': 10000000000,
 'cumulativeStepUsed': 160840,
 'eventLogs': [{'scoreAddress': 'cx19be1be7b4e7750863edf966faac3f42dec21025',
   'indexed': ['Transfer(Address,Address,int,bytes)',
    'hx91bf040426f226b3bfcd2f0b5967bbb0320525ce',
    'hx5bb03cc3f3d26b2d27491c2f56d233ed6777a654',
    '0x1b1ae4d6e2ef500000'],
   'data': ['0x4e6f6e65']}],
 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00 \x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00P\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x80\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00@\x00\x00\x00\x00\x00\x00\

### Function to switch wallets to run transaction

In [103]:
wallets={'1':deployer_wallet1,'2':deployer_wallet2,'3':deployer_wallet3, '4': deployer_wallet4 ,'5': deployer_wallet5}
def wallet_selector():
    wallet=input("Enter the wallet from which you want to run the transaction")
    if str(wallet) not in wallets.keys():
        print("invalid wallet selected")
        return null
    return wallets[str(wallet)]


# Depositing USDb from a wallet

In [67]:
deployer_wallet=wallet_selector()
data = "{\"method\": \"deposit\", \"params\": {\"amount\": 1000000000000000000000}}".encode("utf-8")
params = {"_to": contracts['lendingPool']['SCORE'], "_value": 1000000000000000000000, "_data": data}
transaction = CallTransactionBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['sample_token']['SCORE'])\
    .value(0)\
    .step_limit(10000000)\
    .nid(3)\
    .nonce(100)\
    .method("transfer")\
    .params(params)\
    .build()
signed_transaction = SignedTransaction(transaction, deployer_wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
get_tx_result(tx_hash)

Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...


{'txHash': '0xb1fc5062c7d5fa859a08dd5b505b322bd03450067aec12f06e456c79c25c7d26',
 'blockHeight': 443726,
 'blockHash': '0x1bcfa6746677749da2e16fb7b42ff3708e2dcb3a9fcb7f6a35458f58c72ea4e6',
 'txIndex': 1,
 'to': 'cx2c002610b8aeaff84f6cb97c28294b1986016143',
 'stepUsed': 729440,
 'stepPrice': 10000000000,
 'cumulativeStepUsed': 729440,
 'eventLogs': [{'scoreAddress': 'cx879e276135a6f25236a8abf2e4c9801887538f60',
   'indexed': ['ReserveUpdated(Address,int,int,int,int)',
    'cx2c002610b8aeaff84f6cb97c28294b1986016143',
    '0x0',
    '0x0'],
   'data': ['0xde0b6b3a7640000', '0xde0b6b3a7640000']},
  {'scoreAddress': 'cx2f9c6ae40e6ae4c8d39961b3f3d701363b361773',
   'indexed': ['Mint(Address,int)',
    'hx91bf040426f226b3bfcd2f0b5967bbb0320525ce'],
   'data': ['0x0']},
  {'scoreAddress': 'cx2f9c6ae40e6ae4c8d39961b3f3d701363b361773',
   'indexed': ['Mint(Address,int)',
    'hx91bf040426f226b3bfcd2f0b5967bbb0320525ce'],
   'data': ['0x3635c9adc5dea00000']},
  {'scoreAddress': 'cx2f9c6ae40e6ae4

In [26]:
deployer_wallet=wallet_selector()
params ={"_reserveAddress": contracts['sicx']['SCORE'], "_baseLTVasCollateral":5*10**17 }
call_transaction = CallTransactionBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['lendingPoolCore']['SCORE']) \
    .nid(NID) \
    .nonce(100) \
    .method("updateBaseLTVasCollateral")\
    .params(params)\
    .build()

estimate_step = icon_service.estimate_step(call_transaction)
step_limit = estimate_step + 100000
# Returns the signed transaction object having a signature
signed_transaction = SignedTransaction(call_transaction, deployer_wallet,step_limit)

# Reads params to transfer to nodes
print(signed_transaction.signed_transaction_dict)

# Sends the transaction
tx_hash = icon_service.send_transaction(signed_transaction)
get_tx_result(tx_hash)

Enter the wallet from which you want to run the transaction1
{'version': '0x3', 'from': 'hx91bf040426f226b3bfcd2f0b5967bbb0320525ce', 'to': 'cx327a386a9f8076045679fc132a8baa9e270ca43b', 'stepLimit': '0x3ea80', 'timestamp': '0x5b61ac947ddec', 'nid': '0x3', 'nonce': '0x64', 'dataType': 'call', 'data': {'method': 'updateBaseLTVasCollateral', 'params': {'_reserveAddress': 'cx3e9d38175d112cd5cf05a90080cce779f202ba27', '_baseLTVasCollateral': '0x6f05b59d3b20000'}}, 'signature': 'oJiiqWD8z84P5L8NRI+07ZT0DuSvlnJBliXpuzCJ+2cIE9CAmqdY5sHMy/8tvbfnAXGugQBUF9Q7SHdyM19K5wE='}


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 2 seconds...


{'txHash': '0xe97345936a9e5071f79566ba7f55da791dff825b728f7f2653b2a732fc3d6528',
 'blockHeight': 10280234,
 'blockHash': '0x96c7952c976b62f1fb704a6cba63cebc5f523854133fe84fd3356c833e7fba80',
 'txIndex': 1,
 'to': 'cx327a386a9f8076045679fc132a8baa9e270ca43b',
 'stepUsed': 156640,
 'stepPrice': 10000000000,
 'cumulativeStepUsed': 156640,
 'eventLogs': [],
 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00

## Price oracle

In [107]:
deployer_wallet=wallet_selector()
params ={"_base": 'sICX', "_quote":"USD" , "_rate": 37 * 10**16}
call_transaction = CallTransactionBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['priceOracle']['SCORE']) \
    .nid(NID) \
    .nonce(100) \
    .method("set_reference_data")\
    .params(params)\
    .build()

estimate_step = icon_service.estimate_step(call_transaction)
step_limit = estimate_step + 100000
# Returns the signed transaction object having a signature
signed_transaction = SignedTransaction(call_transaction, deployer_wallet,step_limit)

# Reads params to transfer to nodes
print(signed_transaction.signed_transaction_dict)

# Sends the transaction
tx_hash = icon_service.send_transaction(signed_transaction)
get_tx_result(tx_hash)

Enter the wallet from which you want to run the transaction1
{'version': '0x3', 'from': 'hx91bf040426f226b3bfcd2f0b5967bbb0320525ce', 'to': 'cx3f6477bf290eb91257b806ece614ab18b1c7be41', 'stepLimit': '0x3bf88', 'timestamp': '0x5b61ef8b7e91e', 'nid': '0x3', 'nonce': '0x64', 'dataType': 'call', 'data': {'method': 'set_reference_data', 'params': {'_base': 'sICX', '_quote': 'USD', '_rate': '0x522810a26e50000'}}, 'signature': '9rPT79Jgnocbv41QJVr6WsYTLvxlhl2x5R5x4REMy692Dm6Q+Lt03Vi0nu9QG9Fs6p0G/olqz5TmeuWiKInxNQA='}


{'txHash': '0x9ca746e297876d518bdb7491fcde723b64b14e8e2e4e695b0c77b8b619767b15',
 'blockHeight': 10289193,
 'blockHash': '0x48c8914d7c3a68f644044b0c50f42fc868a59a35c8e9b84ea268f3a0e2ab9d9f',
 'txIndex': 1,
 'to': 'cx3f6477bf290eb91257b806ece614ab18b1c7be41',
 'stepUsed': 145640,
 'stepPrice': 10000000000,
 'cumulativeStepUsed': 145640,
 'eventLogs': [],
 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00

In [ ]:
## Fee Provider

In [32]:
deployer_wallet=wallet_selector()
params ={"_percentage": 10**16}
call_transaction = CallTransactionBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['feeProvider']['SCORE']) \
    .nid(NID) \
    .nonce(100) \
    .method("setLoanOriginationFeePercentage")\
    .params(params)\
    .build()

estimate_step = icon_service.estimate_step(call_transaction)
step_limit = estimate_step + 100000
# Returns the signed transaction object having a signature
signed_transaction = SignedTransaction(call_transaction, deployer_wallet,step_limit)

# Reads params to transfer to nodes
print(signed_transaction.signed_transaction_dict)

# Sends the transaction
tx_hash = icon_service.send_transaction(signed_transaction)
get_tx_result(tx_hash)

Enter the wallet from which you want to run the transaction1
{'version': '0x3', 'from': 'hx91bf040426f226b3bfcd2f0b5967bbb0320525ce', 'to': 'cxede316aa65b7df13633599e67490daf1fd32b8a5', 'stepLimit': '0x3b5d8', 'timestamp': '0x5b61d03eb5248', 'nid': '0x3', 'nonce': '0x64', 'dataType': 'call', 'data': {'method': 'setLoanOriginationFeePercentage', 'params': {'_percentage': '0x2386f26fc10000'}}, 'signature': 'QkxRhUdlpOGzgVM6muwJNSpc8PK24dvqlJLHaOqVIe1ZVS8FNl+poKrk9q2eG/EKwFBx6wNbR2UAIP02D3B5ZwE='}


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...


{'txHash': '0x30f45170afe1a74b1f776265d8017c7e98fb8a1cbf5158a029ad251f6343ed29',
 'blockHeight': 10285005,
 'blockHash': '0x451f05db22966b2beb0c0937294842d181015656f74f2248efc2b4df651ae31e',
 'txIndex': 1,
 'to': 'cxede316aa65b7df13633599e67490daf1fd32b8a5',
 'stepUsed': 143160,
 'stepPrice': 10000000000,
 'cumulativeStepUsed': 143160,
 'eventLogs': [],
 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00

In [33]:

_call = CallBuilder()\
    .from_(deployer_wallet1.get_address())\
    .to(contracts['feeProvider']['SCORE'])\
    .method("getLoanOriginationFeePercentage")\
    .build()
response = icon_service.call(_call)


pprint(response)

'0x2386f26fc10000'


# Borrowing USDb

In [106]:
deployer_wallet=wallet_selector()
params ={"_reserve": contracts['sample_token']['SCORE'], "_amount":75*10**18 }
call_transaction = CallTransactionBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['lendingPool']['SCORE']) \
    .nid(NID) \
    .nonce(100) \
    .method("borrow")\
    .params(params)\
    .build()

estimate_step = icon_service.estimate_step(call_transaction)
step_limit = estimate_step + 100000
# Returns the signed transaction object having a signature
signed_transaction = SignedTransaction(call_transaction, deployer_wallet,step_limit)

# Reads params to transfer to nodes
print(signed_transaction.signed_transaction_dict)

# Sends the transaction
tx_hash = icon_service.send_transaction(signed_transaction)
get_tx_result(tx_hash)

Enter the wallet from which you want to run the transaction4
{'version': '0x3', 'from': 'hxd4ff18afb94cd6cb41d8ff468254112619a19051', 'to': 'cx35b12ca4e9c672518db3c06d92ed60d50e7d5554', 'stepLimit': '0x148264', 'timestamp': '0x5b61ef7d1a27e', 'nid': '0x3', 'nonce': '0x64', 'dataType': 'call', 'data': {'method': 'borrow', 'params': {'_reserve': 'cx19be1be7b4e7750863edf966faac3f42dec21025', '_amount': '0x410d586a20a4c0000'}}, 'signature': 'E5880fcAsg9Nz89K7kBA+l2jUyE+oeiu+ydXbcn8u1UjXTWLw4DTJrX9aTdII1fdXoDYh7yhGsSWQdbxRHGNAAE='}


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 2 seconds...


{'txHash': '0x12e589d7edcaf079718bcbba853871d2ce25c3a80eb54e2a8bb0bc6263bebf9a',
 'blockHeight': 10289184,
 'blockHash': '0x66f4802b79628bca12060ae587a5fe85fc242697c7d0ccc2bb34d9d5dccd4dcd',
 'txIndex': 1,
 'to': 'cx35b12ca4e9c672518db3c06d92ed60d50e7d5554',
 'stepUsed': 1244100,
 'stepPrice': 10000000000,
 'cumulativeStepUsed': 1244100,
 'eventLogs': [{'scoreAddress': 'cx35b12ca4e9c672518db3c06d92ed60d50e7d5554',
   'indexed': ['PrintData(str,int,int,int)',
    'available liquidity at pool line 245',
    '0x5cdaf8e257da9a77be1',
    '0x0'],
   'data': ['0x0']},
  {'scoreAddress': 'cx35b12ca4e9c672518db3c06d92ed60d50e7d5554',
   'indexed': ['PrintData(str,int,int,int)',
    'data at pool line 256',
    '0x821ab0d450440cd73',
    '0x0'],
   'data': ['0x0']},
  {'scoreAddress': 'cx35b12ca4e9c672518db3c06d92ed60d50e7d5554',
   'indexed': ['PrintData(str,int,int,int)',
    'amout of collateral needed USD pool line 268',
    '0x821ab0d4414980000',
    '0x0'],
   'data': ['0x0']},
  {'scoreA

# Reedeming/Withdrawing USDb

In [ ]:
deployer_wallet=wallet_selector()
params ={"_amount": 10*10**18 }
call_transaction = CallTransactionBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['oToken']['SCORE']) \
    .nid(NID) \
    .nonce(100) \
    .method("redeem")\
    .params(params)\
    .build()

estimate_step = icon_service.estimate_step(call_transaction)
step_limit = estimate_step + 100000
# Returns the signed transaction object having a signature
signed_transaction = SignedTransaction(call_transaction, deployer_wallet,step_limit)

# Reads params to transfer to nodes
print(signed_transaction.signed_transaction_dict)

# Sends the transaction
tx_hash = icon_service.send_transaction(signed_transaction)
get_tx_result(tx_hash)

# Repaying USDb

In [ ]:
deployer_wallet=wallet_selector()
data = "{\"method\": \"repay\", \"params\": {\"amount\": 100000000000000000000}}".encode("utf-8")
params = {"_to": contracts['lendingPool']['SCORE'], "_value": 100000000000000000000, "_data": data}
transaction = CallTransactionBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['sicx']['SCORE'])\
    .value(0)\
    .step_limit(10000000)\
    .nid(3)\
    .nonce(100)\
    .method("transfer")\
    .params(params)\
    .build()
signed_transaction = SignedTransaction(transaction, deployer_wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
get_tx_result(tx_hash)

# Depositing ICX

In [105]:
deployer_wallet=wallet_selector()
params = {"_amount": 300 * 10 ** 18}
call_transaction = CallTransactionBuilder() \
    .from_(deployer_wallet.get_address()) \
    .to(contracts['lendingPool']['SCORE']) \
    .nid(3) \
    .step_limit(1000000000) \
    .nonce(100) \
    .value(300 * 10 ** 18) \
    .method("deposit") \
    .params(params) \
    .build() 

signed_transaction = SignedTransaction(call_transaction, deployer_wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
get_tx_result(tx_hash)

Enter the wallet from which you want to run the transaction4


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 2 seconds...


{'txHash': '0x00d4d565300f57271dac3163e6e507377852c7dc0b75389aff6ea7eecd83e119',
 'blockHeight': 10289175,
 'blockHash': '0xaac659a5dc92ba0b2753ed0df59e4839cb807c2575a050f952b247f230717ac6',
 'txIndex': 1,
 'to': 'cx35b12ca4e9c672518db3c06d92ed60d50e7d5554',
 'stepUsed': 574400,
 'stepPrice': 10000000000,
 'cumulativeStepUsed': 574400,
 'eventLogs': [{'scoreAddress': 'cx35b12ca4e9c672518db3c06d92ed60d50e7d5554',
   'indexed': ['ICXTransfer(Address,Address,int)',
    'cx35b12ca4e9c672518db3c06d92ed60d50e7d5554',
    'cx3e9d38175d112cd5cf05a90080cce779f202ba27',
    '0x1043561a8829300000'],
   'data': []},
  {'scoreAddress': 'cx3e9d38175d112cd5cf05a90080cce779f202ba27',
   'indexed': ['Mint(Address,int)',
    'cx327a386a9f8076045679fc132a8baa9e270ca43b',
    '0x1043561a8829300000'],
   'data': []},
  {'scoreAddress': 'cx327a386a9f8076045679fc132a8baa9e270ca43b',
   'indexed': ['PrintData(str,int,int,int)',
    'params check core line 408',
    '0x1043561a8829300000',
    '0x0'],
   'data

# Withdrawing ICX 

In [ ]:
deployer_wallet=wallet_selector()
params = {"_amount": 250 * 10 ** 18}
call_transaction = CallTransactionBuilder() \
    .from_(deployer_wallet.get_address())\
    .to(contracts['oICX']['SCORE'])\
    .nid(3) \
    .step_limit(10000000) \
    .nonce(100) \
    .method("redeem") \
    .params(params) \
    .build()
signed_transaction = SignedTransaction(call_transaction, deployer_wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
get_tx_result(tx_hash)

# Borrow sICX

In [ ]:
deployer_wallet=wallet_selector()
params ={"_reserve": contracts['sicx']['SCORE'], "_amount":150 * 10 ** 18 }
call_transaction = CallTransactionBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['lendingPool']['SCORE']) \
    .nid(NID) \
    .nonce(100) \
    .method("borrow")\
    .params(params)\
    .build()

estimate_step = icon_service.estimate_step(call_transaction)
step_limit = estimate_step + 100000
# Returns the signed transaction object having a signature
signed_transaction = SignedTransaction(call_transaction, deployer_wallet,step_limit)

# Reads params to transfer to nodes
print(signed_transaction.signed_transaction_dict)

# Sends the transaction
tx_hash = icon_service.send_transaction(signed_transaction)
get_tx_result(tx_hash)

# Repay sICX

In [ ]:
deployer_wallet=wallet_selector()
data = "{\"method\": \"repay\", \"params\": {\"amount\": 100 * 10 ** 18}}".encode("utf-8")
params = {"_to": contracts['lendingPool']['SCORE'], "_value": 100 * 10 ** 18, "_data": data}
transaction = CallTransactionBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['sicx']['SCORE'])\
    .value(0)\
    .step_limit(10000000)\
    .nid(3)\
    .nonce(100)\
    .method("transfer")\
    .params(params)\
    .build()
signed_transaction = SignedTransaction(transaction, deployer_wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
get_tx_result(tx_hash)

## Liquidation call

In [111]:
#Specify the wallet of the borrower
_borrowerUser = wallet_selector()
#Specify the wallet with which you wish to repay _borrowUser's borrowed asset
_liquidatorUser = deployer_wallet5
_loanPayoffAmount = 25*10**18
depositData = {'method': 'liquidationCall', 'params': 
            {'_collateral': contracts['sicx']["SCORE"],
            '_reserve': contracts['sample_token']["SCORE"],
            '_user': _borrowerUser.get_address(),
            '_purchaseAmount': _loanPayoffAmount}}
data = json.dumps(depositData).encode('utf-8')
params = {"_to": contracts['lendingPool']['SCORE'],
            "_value": _loanPayoffAmount, "_data": data}
call_transaction = CallTransactionBuilder() \
    .from_(_liquidatorUser.get_address()) \
    .to(contracts['sample_token']["SCORE"]) \
    .nid(3) \
    .step_limit(10000000) \
    .nonce(100) \
    .method("transfer") \
    .params(params) \
    .build()
signed_transaction = SignedTransaction(call_transaction, _liquidatorUser)
tx_hash = icon_service.send_transaction(signed_transaction)
get_tx_result(tx_hash)

Enter the wallet from which you want to run the transaction4


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 2 seconds...


{'txHash': '0xb2b13e9d92658e21568e77cbe320a2cd2c16885e13b549e3cca97bda428ea5e5',
 'blockHeight': 10289239,
 'blockHash': '0xba517c7ad46b2fea6125e7deb2f499b147e5aa6bb68d6d775e243b6741d1dd4d',
 'txIndex': 1,
 'to': 'cx19be1be7b4e7750863edf966faac3f42dec21025',
 'stepUsed': 2720060,
 'stepPrice': 10000000000,
 'cumulativeStepUsed': 2720060,
 'eventLogs': [{'scoreAddress': 'cx327a386a9f8076045679fc132a8baa9e270ca43b',
   'indexed': ['PrintData(str,int,int,int)',
    'params check core line 408',
    '0x1190676589db839a6',
    '0x0'],
   'data': ['0x0']},
  {'scoreAddress': 'cx327a386a9f8076045679fc132a8baa9e270ca43b',
   'indexed': ['PrintData(str,int,int,int)',
    'rate value core line 412',
    '0x91163baed9fb',
    '0x2751fd2e2fa6de'],
   'data': ['0x0']},
  {'scoreAddress': 'cx327a386a9f8076045679fc132a8baa9e270ca43b',
   'indexed': ['ReserveUpdated(Address,int,int,int,int)',
    'cx19be1be7b4e7750863edf966faac3f42dec21025',
    '0x91163baed9fb',
    '0x2751fd2e2fa6de'],
   'data': ['

In [54]:
icon_service.get_balance(deployer_wallet5.get_address())

8399972201400000000000

In [61]:
params = {'_owner': deployer_wallet5.get_address()}

_call = CallBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['sample_token']["SCORE"])\
    .method("balanceOf")\
    .params(params)\
    .build()
response = icon_service.call(_call)
pprint(response)

'0x1e084b60c83e680000'


# Readonly methods to get data

In [108]:
deployer_wallet=wallet_selector()
params = {'_user': deployer_wallet.get_address()}
_call = CallBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['lendingPoolDataProvider']['SCORE'])\
    .method("getUserAccountData")\
    .params(params)\
    .build()
response = icon_service.call(_call)
for key,values in response.items():
    if key!="lastUpdateTimestamp":
        response[key]=int(values,0)/10**18
    else:
        response[key]=int(v,0)
        
pprint(response)

Enter the wallet from which you want to run the transaction4
{'borrowingPower': 1.049896060813527,
 'currentLiquidationThreshold': 0.65,
 'currentLtv': 0.5,
 'healthFactor': 0.9524752376203179,
 'healthFactorBelowThreshold': 1e-18,
 'totalBorrowBalanceUSD': 75.00000079311067,
 'totalCollateralBalanceUSD': 111.00000000793442,
 'totalFeesUSD': 0.75,
 'totalLiquidityBalanceUSD': 111.00000000793442}


In [112]:
deployer_wallet=wallet_selector()
params = {'_user': deployer_wallet.get_address()}
_call = CallBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['lendingPoolDataProvider']['SCORE'])\
    .method("getUserAllReserveData")\
    .params(params)\
    .build()
response = icon_service.call(_call)
for key,values in response.items():
    for k,v in values.items():
        if k!="lastUpdateTimestamp":
            response[key][k]=int(v,0)/10**18
        else:
            response[key][k]=int(v,0)
pprint(response)

Enter the wallet from which you want to run the transaction4
{'USDb': {'borrowRate': 0.011067671935166175,
          'currentBorrowBalance': 54.750000471732214,
          'currentBorrowBalanceUSD': 54.750000471732214,
          'currentOTokenBalance': 0.0,
          'currentOTokenBalanceUSD': 0.0,
          'lastUpdateTimestamp': 1607619117869737,
          'liquidityRate': 0.000159524676622843,
          'originationFee': 0.0,
          'principalBorrowBalance': 54.750000010579214,
          'principalBorrowBalanceUSD': 54.750000010579214,
          'useAsCollateral': 0.0,
          'userBorrowCumulativeIndex': 1.0003589628882579},
 'sICX': {'borrowRate': 0.00246734981663832,
          'currentBorrowBalance': 0.0,
          'currentBorrowBalanceUSD': 0.0,
          'currentOTokenBalance': 237.56755905245768,
          'currentOTokenBalanceUSD': 87.89999684940933,
          'lastUpdateTimestamp': 0,
          'liquidityRate': 4.6963145193417e-05,
          'originationFee': 0.0,
      

In [97]:

deployer_wallet=wallet_selector()
params = {'_reserve': contracts['sicx']['SCORE']}
_call = CallBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['lendingPoolCore']['SCORE'])\
    .method("getReserveConfiguration")\
    .params(params)\
    .build()
response = icon_service.call(_call)

pprint(response)

Enter the wallet from which you want to run the transaction1
{'baseLTVasCollateral': '0x6f05b59d3b20000',
 'borrowingEnabled': '0x1',
 'decimals': '0x12',
 'isActive': '0x1',
 'liquidationBonus': '0x16345785d8a0000',
 'liquidationThreshold': '0x905438e60010000',
 'usageAsCollateralEnabled': '0x1'}


## Getting specific reserve data

### This returns the reserve data of USDb

In [87]:
params = {'_reserve': contracts['sample_token']['SCORE']}
_call = CallBuilder()\
    .from_(deployer_wallet1.get_address())\
    .to(contracts['lendingPoolDataProvider']['SCORE'])\
    .method("getReserveData")\
    .params(params)\
    .build()
response = icon_service.call(_call)
for key,values in response.items():
    if values=="0x1" or values =="0x0" or "Address" in key:
        continue
    if key!="lastUpdateTimestamp":
        response[key]=int(values,0)/10**18
    else:
        response[key]=int(values,0)

pprint(response)

{'availableLiquidity': 27385.815034154173,
 'baseLTVasCollateral': 0.5,
 'borrowCumulativeIndex': 1.000358823081847,
 'borrowRate': 0.010984927638586547,
 'borrowingEnabled': '0x1',
 'decimals': 1.8e-17,
 'isActive': '0x1',
 'isFreezed': '0x0',
 'lastUpdateTimestamp': 1607618407851662,
 'liquidationBonus': 0.1,
 'liquidationThreshold': 0.65,
 'liquidityCumulativeIndex': 1.00000411761864,
 'liquidityRate': 0.000146061344328081,
 'oTokenAddress': 'cx4258464e395b3ef3b73a146947eb41393e8c5873',
 'reserveAddress': 'cx19be1be7b4e7750863edf966faac3f42dec21025',
 'totalBorrows': 410.66278894999925,
 'totalLiquidity': 27796.47782310417,
 'usageAsCollateralEnabled': '0x1'}


#### This method returns the reserve data of sICX

In [24]:
params = {'_reserve': contracts['sicx']['SCORE']}
_call = CallBuilder()\
    .from_(deployer_wallet1.get_address())\
    .to(contracts['lendingPoolDataProvider']['SCORE'])\
    .method("getReserveData")\
    .params(params)\
    .build()
response = icon_service.call(_call)
for key,values in response.items():
    if values=="0x1" or values =="0x0" or "Address" in key:
        continue
    if key!="lastUpdateTimestamp":
        response[key]=int(values,0)/10**18
    else:
        response[key]=int(values,0)

pprint(response)

{'availableLiquidity': '0x0',
 'baseLTVasCollateral': 0.33,
 'borrowCumulativeIndex': 1.0,
 'borrowRate': '0x0',
 'borrowingEnabled': '0x1',
 'decimals': 1.8e-17,
 'isActive': '0x1',
 'isFreezed': '0x0',
 'lastUpdateTimestamp': '0x0',
 'liquidationBonus': 1e-17,
 'liquidationThreshold': 0.65,
 'liquidityCumulativeIndex': 1.0,
 'liquidityRate': '0x0',
 'oTokenAddress': 'cxe6b279229d10c9230b59cb50349589a015401925',
 'reserveAddress': 'cxe74650f0903ef43102b74fb7d4949ad2f42eeff9',
 'totalBorrows': '0x0',
 'totalLiquidity': '0x0',
 'usageAsCollateralEnabled': '0x1'}


## Getting data of all reserves

### This returns the reserve data of all the reserves

In [16]:
params = {'_reserve': contracts['sample_token']['SCORE']}
_call = CallBuilder()\
    .from_(deployer_wallet1.get_address())\
    .to(contracts['lendingPoolDataProvider']['SCORE'])\
    .method("getAllReserveData")\
    .build()
response = icon_service.call(_call)
for key,values in response.items():
    for k,v in values.items():
        if v=="0x1" or v =="0x0" or "Address" in k:
            continue
        if k!="lastUpdateTimestamp":
            response[key][k]=int(v,0)/10**18
        else:
            response[key][k]=int(v,0)
pprint(response)

{'USDb': {'availableLiquidity': 27590.564997924317,
          'baseLTVasCollateral': 0.33,
          'borrowCumulativeIndex': 1.000273798950921,
          'borrowRate': 0.010493858227441847,
          'borrowingEnabled': '0x1',
          'decimals': 1.8e-17,
          'isActive': '0x1',
          'isFreezed': '0x0',
          'lastUpdateTimestamp': 1607361908231307,
          'liquidationBonus': 1e-17,
          'liquidationThreshold': 0.65,
          'liquidityCumulativeIndex': 1.0000035341626332,
          'liquidityRate': 6.9963456013611e-05,
          'oTokenAddress': 'cx4258464e395b3ef3b73a146947eb41393e8c5873',
          'reserveAddress': 'cx19be1be7b4e7750863edf966faac3f42dec21025',
          'totalBorrows': 205.91278873267638,
          'totalLiquidity': 27796.477786656993,
          'usageAsCollateralEnabled': '0x1'},
 'sICX': {'availableLiquidity': 240.49,
          'baseLTVasCollateral': 0.33,
          'borrowCumulativeIndex': 1.000320647242915,
          'borrowRate': 0.01

### Getting user account data

In [ ]:
deployer_wallet=wallet_selector()
params = {'_user': deployer_wallet.get_address()}
_call = CallBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['lendingPoolDataProvider']['SCORE'])\
    .method("getUserAccountData")\
    .params(params)\
    .build()
response = icon_service.call(_call)
for key,values in response.items():
    if key!="lastUpdateTimestamp":
        response[key]=int(values,0)/10**18
    else:
        response[key]=int(v,0)

### Returns the user data of all the reserves combined

In [ ]:
deployer_wallet=wallet_selector()
params = {'_user': deployer_wallet.get_address()}
_call = CallBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['lendingPoolDataProvider']['SCORE'])\
    .method("getUserAllReserveData")\
    .params(params)\
    .build()
response = icon_service.call(_call)
for key,values in response.items():
    for k,v in values.items():
        if k!="lastUpdateTimestamp":
            response[key][k]=int(v,0)/10**18
        else:
            response[key][k]=int(v,0)
pprint(response)

### Getting user reserve data for USDb reserve

In [15]:
deployer_wallet=wallet_selector()
params = {'_user': deployer_wallet.get_address() ,"_reserve":contracts['sample_token']['SCORE']}
_call = CallBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['lendingPoolDataProvider']['SCORE'])\
    .method("getUserReserveData")\
    .params(params)\
    .build()
response = icon_service.call(_call)
for key,values in response.items():
    if key!="lastUpdateTimestamp":
        response[key]=int(values,0)/10**18
    else:
        response[key]=int(values,0)
pprint(response)

{'borrowRate': 0.0,
 'currentBorrowBalance': 0.0,
 'currentBorrowBalanceUSD': 0.0,
 'currentOTokenBalance': 0.0,
 'currentOTokenBalanceUSD': 0.0,
 'lastUpdateTimestamp': 0,
 'liquidityRate': 0.0,
 'originationFee': 0.0,
 'principalBorrowBalance': 0.0,
 'principalBorrowBalanceUSD': 0.0,
 'useAsCollateral': 0.0,
 'userBorrowCumulativeIndex': 0.0}


### Getting user reserve data for sICX reserve

In [16]:
deployer_wallet=wallet_selector()
params = {'_user': deployer_wallet.get_address() ,"_reserve":contracts['sample_token']['SCORE']}
_call = CallBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['lendingPoolDataProvider']['SCORE'])\
    .method("getUserReserveData")\
    .params(params)\
    .build()
response = icon_service.call(_call)
for key,values in response.items():
    if key!="lastUpdateTimestamp":
        response[key]=int(values,0)/10**18
    else:
        response[key]=int(values,0)
pprint(response)

{'borrowRate': 0.0,
 'currentBorrowBalance': 0.0,
 'currentBorrowBalanceUSD': 0.0,
 'currentOTokenBalance': 0.0,
 'currentOTokenBalanceUSD': 0.0,
 'lastUpdateTimestamp': 0,
 'liquidityRate': 0.0,
 'originationFee': 0.0,
 'principalBorrowBalance': 0.0,
 'principalBorrowBalanceUSD': 0.0,
 'useAsCollateral': 0.0,
 'userBorrowCumulativeIndex': 0.0}


### Getting the data of user for all reserves

In [62]:
deployer_wallet=wallet_selector()
params = {'_user': deployer_wallet.get_address()}
_call = CallBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['lendingPoolDataProvider']['SCORE'])\
    .method("getUserAllReserveData")\
    .params(params)\
    .build()
response = icon_service.call(_call)
for key,values in response.items():
    for k,v in values.items():
        if k!="lastUpdateTimestamp":
            response[key][k]=int(v,0)/10**18
        else:
            response[key][k]=int(v,0)
pprint(response)

Enter the wallet from which you want to run the transaction4
{'USDb': {'borrowRate': 0.010805048733359064,
          'currentBorrowBalance': 54.7500433812586,
          'currentBorrowBalanceUSD': 54.7500433812586,
          'currentOTokenBalance': 0.0,
          'currentOTokenBalanceUSD': 0.0,
          'lastUpdateTimestamp': 1607612431329189,
          'liquidityRate': 0.000117430975755095,
          'originationFee': 0.0,
          'principalBorrowBalance': 54.75000021732289,
          'principalBorrowBalanceUSD': 54.75000021732289,
          'useAsCollateral': 0.0,
          'userBorrowCumulativeIndex': 1.0003567748171815},
 'sICX': {'borrowRate': 0.003267840664224172,
          'currentBorrowBalance': 0.0,
          'currentBorrowBalanceUSD': 0.0,
          'currentOTokenBalance': 243.24314869173278,
          'currentOTokenBalanceUSD': 89.99996501594113,
          'lastUpdateTimestamp': 0,
          'liquidityRate': 8.2379180109269e-05,
          'originationFee': 0.0,
          '

## Get reserve configurations


### Getting configuration of USDb reserve

In [20]:
params = {"_reserve":contracts['sample_token']['SCORE']}
_call = CallBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['lendingPoolDataProvider']['SCORE'])\
    .method("getReserveConfigurationData")\
    .params(params)\
    .build()
response = icon_service.call(_call)
for key,values in response.items():
    response[key]=int(values,0)
pprint(response)


{'baseLTVasCollateral': 600000000000000000,
 'borrowingEnabled': 1,
 'decimals': 18,
 'isActive': 1,
 'liquidationBonus': 10,
 'liquidationThreshold': 650000000000000000,
 'usageAsCollateralEnabled': 1}


### Getting configuration of sICX reserve

In [17]:
params = {"_reserve":contracts['sicx']['SCORE']}
_call = CallBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['lendingPoolDataProvider']['SCORE'])\
    .method("getReserveConfigurationData")\
    .params(params)\
    .build()
response = icon_service.call(_call)
for key,values in response.items():
    response[key]=int(values,0)
pprint(response)

{'baseLTVasCollateral': 330000000000000000,
 'borrowingEnabled': 1,
 'decimals': 18,
 'isActive': 1,
 'liquidationBonus': 10,
 'liquidationThreshold': 650000000000000000,
 'usageAsCollateralEnabled': 1}


### Get all reserves configurations

In [18]:
_call = CallBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['lendingPoolDataProvider']['SCORE'])\
    .method("getAllReserveConfigurationData")\
    .build()
response = icon_service.call(_call)
for key,values in response.items():
    for k,v in values.items():
        response[key][k]=int(v,0)
pprint(response)


{'USDb': {'baseLTVasCollateral': 600000000000000000,
          'borrowingEnabled': 1,
          'decimals': 18,
          'isActive': 1,
          'liquidationBonus': 10,
          'liquidationThreshold': 650000000000000000,
          'usageAsCollateralEnabled': 1},
 'sICX': {'baseLTVasCollateral': 330000000000000000,
          'borrowingEnabled': 1,
          'decimals': 18,
          'isActive': 1,
          'liquidationBonus': 10,
          'liquidationThreshold': 650000000000000000,
          'usageAsCollateralEnabled': 1}}


### Printing ICX balance from respective wallets

In [7]:
print('ICX balance of deployer_wallet3',icon_service.get_balance(deployer_wallet3.get_address()))
print('ICX balance of deployer_wallet1', icon_service.get_balance(deployer_wallet1.get_address()))
print('ICX balance of deployer_wallet2',icon_service.get_balance(deployer_wallet2.get_address()))  

ICX balance of deployer_wallet3 999999998408600000000000
ICX balance of deployer_wallet1 78702702844323400000000000
ICX balance of deployer_wallet2 999999998402400000000000


## Realtime USDb balance of user(with accrued interest)

In [10]:
deployer_wallet=wallet_selector()
params = {'_owner': deployer_wallet.get_address() }
_call = CallBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['oToken']['SCORE'])\
    .method("balanceOf")\
    .params(params)\
    .build()
response = icon_service.call(_call)

print(int(response,0))

0


## Realtime ICX balance of user(with accured interest)

In [21]:
deployer_wallet=wallet_selector()
params = {'_owner': deployer_wallet.get_address() }
_call = CallBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['oICX']['SCORE'])\
    .method("balanceOf")\
    .params(params)\
    .build()
response = icon_service.call(_call)

print(int(response,0))


0


### Printing USDb balance from respective wallets

In [11]:
 wallets = [deployer_wallet1, deployer_wallet2, deployer_wallet3]
 for wallet in wallets:
    params = {'_owner': wallet.get_address()}
    _call = CallBuilder() \
        .from_(wallet.get_address()) \
        .to(contracts['sample_token']['SCORE']) \
        .method("balanceOf") \
        .params(params) \
        .build()
    response = icon_service.call(_call)
    print('USDb balance of ', wallet.get_address(), 'is' ,int(response,16))   

USDb balance of  hx91bf040426f226b3bfcd2f0b5967bbb0320525ce is 250000000000000000000000000
USDb balance of  hx8c8ccc79c99375de5995ca6057cda7c334d9dd35 is 125000000000000000000000000
USDb balance of  hx49c8a4c31e84b9abe7f8c8395eb7133b2151aeae is 125000000000000000000000000


### Printing sICX balance from respective wallets

In [12]:
 wallets = [deployer_wallet1, deployer_wallet2, deployer_wallet3]
 for wallet in wallets:
    params = {'_owner': wallet.get_address()}
    _call = CallBuilder() \
        .from_(wallet.get_address()) \
        .to(contracts['sicx']['SCORE']) \
        .method("balanceOf") \
        .params(params) \
        .build()
    response = icon_service.call(_call)
    print('sICX balance of ', wallet.get_address(), 'is' ,int(response,16))   

sICX balance of  hx91bf040426f226b3bfcd2f0b5967bbb0320525ce is 500000000000000000000000000
sICX balance of  hx8c8ccc79c99375de5995ca6057cda7c334d9dd35 is 0
sICX balance of  hx49c8a4c31e84b9abe7f8c8395eb7133b2151aeae is 0
